# Using JuliaFEM

In [ ]:
using JuliaFEM

In [ ]:
meshfile = download("https://github.com/JuliaFEM/JuliaFEM.jl/raw/master/examples/linear_static/JuliaFEMSMP18.med")
mesh = aster_read_mesh(meshfile)

In [ ]:
model = Problem(Elasticity, "OTHER", 3)
model_elements = create_elements(mesh, "OTHER")

In [ ]:
update!(model_elements, "youngs modulus", 208.0E3)
update!(model_elements, "poissons ratio", 0.30)
update!(model_elements, "density", 7.80E-9)
add_elements!(model, model_elements)

In [ ]:
function is_not_Seg3_or_Tri6(element)
    return !isa(element, Union{Element{Seg3}, Element{Tri6}})
end

filter!(is_not_Seg3_or_Tri6, model.elements)

In [ ]:
function add_nodes_at_certain_plane_to_node_set!(mesh, name, vector_id, distance,
                                                 radius=6.0)
    for (node, coords) in mesh.nodes
        if isapprox(coords[vector_id], distance, atol=radius)
            add_node_to_node_set!(mesh, name, node)
        end
    end
    return nothing
end

In [ ]:
add_nodes_at_certain_plane_to_node_set!(mesh, :mid_fixed, 2, 50.0)

In [ ]:
ipoint = find_nearest_nodes(mesh, [165.0, 88.0, 10],3)
for poi in ipoint
    add_node_to_node_set!(mesh, :mid_fixed, poi)
end

In [ ]:
fixed = Problem(Dirichlet, "fixed", 3, "displacement")
fixed_elements = create_nodal_elements(mesh, "mid_fixed")
add_elements!(fixed, fixed_elements)
update!(fixed_elements, "displacement 1", 0.0)
update!(fixed_elements, "displacement 2", 0.0)
update!(fixed_elements, "displacement 3", 0.0)

In [ ]:
update!(model_elements, "displacement load 1", 1.0)

In [ ]:
analysis = Analysis(Linear, model, fixed)

In [ ]:
xdmf = Xdmf("model_results"; overwrite=true)
add_results_writer!(analysis, xdmf)

In [ ]:
push!(model.postprocess_fields, "stress")

In [ ]:
run!(analysis)

In [ ]:
close(xdmf)

In [ ]:
using Test

In [ ]:
@test isfile("model_results.xmf")
@test isfile("model_results.h5")

In [ ]:
using LinearAlgebra
time = 0.0
u = analysis("displacement", time)
u_norms = Dict(i => norm(j) for (i, j) in u)
@test isapprox(maximum(values(u_norms)),2.4052929896922337)